In [1]:
import os
os.chdir('/home/megatron/work')

import megatron
import pandas as pd
import numpy as np
import pickle
import sqlite3
import pymongo
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D
from tensorflow.keras.layers import Dropout, Flatten, Input, Add
from tensorflow.keras import losses, optimizers
from tensorflow.keras.models import Model

In [2]:
inputs = {'X1': Input((48, 48, 1)),
          'X2': Input((48, 48, 1))}
mid_outputs = {}

for input_name, input_node in inputs.items():
    X = Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(48, 48, 1))(input_node)
    X = Conv2D(64, (3, 3), activation='relu')(X)
    X = MaxPooling2D(pool_size=(2, 2))(X)
    X = Dropout(0.25)(X)
    X = Flatten()(X)
    X = Dense(128, activation='relu')(X)
    X = Dropout(0.5)(X)
    X = Dense(2, activation='softmax')(X)
    mid_outputs[input_name] = X
    
output = Add()([mid_outputs['X1'], mid_outputs['X2']])

model = Model(list(inputs.values()), output)

model.compile(loss=losses.categorical_crossentropy,
              optimizer=optimizers.Adadelta(),
              metrics=['accuracy'])

In [3]:
def data_generator(n_obs):
    while True:
        yield {'image1': np.random.random((n_obs, 48, 48, 3)),
               'image2': np.random.random((n_obs, 48, 48, 3)),
               'label': np.random.randint(0, 2, n_obs),
              }

In [4]:
generator = True

if generator:
    data = data_generator(32)
else:
    data = {'image1': np.random.random((1000, 48, 48, 3)),
            'image2': np.random.random((1000, 48, 48, 3)),
            'label': np.random.randint(0, 2, 1000)}

bw_layer = megatron.layers.RGBtoGrey(method='luminosity', keep_dim=True)

image1 = megatron.nodes.Input('image1', shape=(48, 48, 3))
bw_image1 = bw_layer(image1, 'bw_image')

image2 = megatron.nodes.Input('image2', shape=(48, 48, 3))
bw_image2 = bw_layer(image2, 'bw_image')

label_in = megatron.nodes.Input('label')
label = megatron.layers.OneHotRange(max_val=1)(label_in, 'ohe_label')

model_layer = megatron.layers.Keras(model)
pred = model_layer([bw_image1, bw_image2, label], 'predictor')

storage = None if generator else sqlite3.connect('megatron_default')
pipeline = megatron.Pipeline([image1, image2, label_in], pred,
                          name='testkeras', version=0.1,
                          storage=storage)

In [5]:
if generator:
    pipeline.fit_generator(data, steps_per_epoch=5, epochs=2)
    predictions = pipeline.transform_generator(data, steps=5)
else:
    pipeline.fit(data)
    predictions = pipeline.transform(data)

Epoch 1/2
5/5 [==============================] - 1s 286ms/step - loss: 0.7768 - acc: 0.5125
Epoch 2/2
5/5 [==============================] - 1s 229ms/step - loss: 0.6966 - acc: 0.5375


In [ ]:
predictions['predictor'].shape

In [6]:
next(predictions)['predictor'].shape

(32, 2)

In [ ]:
pipeline.storage.read(lookup=['0'])['predictor'].shape

In [ ]:
pipeline.save('test')

In [ ]:
megatron.visuals.pipeline_imsave(pipeline, 'img/keras.png')
megatron.visuals.pipeline_imshow(pipeline)